In [1]:
# libraries

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import json
import os

import re
import spacy 
from spacy import displacy
import gensim
nlp = spacy.load("en_core_web_sm")

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
import string
import contractions
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from pandas_profiling import ProfileReport
import seaborn as sns
import matplotlib.pyplot as plt

from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel

import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder,StandardScaler, RobustScaler

pd.set_option("display.max_columns",None)
pd.set_option("display.width",500)
sns.set(rc={"figure.figsize":(12,12)})

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.width = None
pd.options.display.max_colwidth = None

[nltk_data] Downloading package stopwords to C:\Users\Melis
[nltk_data]     Nur\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Melis
[nltk_data]     Nur\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Define the directory where the JSON files are located
json_dir = 'data/'

# Create an empty list to store the dataframes
df_list = []

# Loop through all files in the directory
for filename in os.listdir(json_dir):
    # Check if the file is a JSON file
    if filename.endswith('.json'):
        # Load the JSON data
        with open(os.path.join(json_dir, filename), 'r', encoding='ISO-8859-1') as f:
            data = json.load(f)
        # Convert the dictionary to a DataFrame and append it to the list
        df_list.append(pd.DataFrame.from_dict(data))

# Concatenate all dataframes in the list into one dataframe
merged_df = pd.concat(df_list, ignore_index=True)

In [3]:
merged_df.head()

,entities,text,id,created_at,public_metrics,author_id,source,edit_history_tweet_ids,referenced_tweets,conversation_id,lang,in_reply_to_user_id,possibly_sensitive,geo,attachments,withheld
0,"{'mentions': [{'start': 16, 'end': 32, 'username': 'ChrisJo80574828', 'id': '1130338903133528064'}, {'start': 33, 'end': 46, 'username': 'TheRealEarns', 'id': '374410249'}, {'start': 47, 'end': 60, 'username': 'MarinaMedvin', 'id': '986232040021549056'}], 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}","@peanut_pitbull @ChrisJo80574828 @TheRealEarns @MarinaMedvin The government has no money they spend more than they receive via taxes. So they borrow from the fed which worsens inflation,worsens wealth inequality, and inflates bubbles. What good are roads etc if you are priced out of purchasing goods, unable to pay rent, and no savings",1457055231225778177,2021-11-06T18:40:12.000Z,"{'retweet_count': 0, 'reply_count': 2, 'like_count': 0, 'quote_count': 0}",1230697110321815552,Twitter for Android,[1457055231225778177],"[{'type': 'replied_to', 'id': '1457002564323401737'}]",1456831355556352003,en,796109962292449280,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN
1,"{'mentions': [{'start': 0, 'end': 15, 'username': 'LitUpMagazine1', 'id': '1216087959503429633'}, {'start': 16, 'end': 32, 'username': 'thomaschattwill', 'id': '133033883'}, {'start': 33, 'end': 46, 'username': 'nhannahjones', 'id': '25598396'}], 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}","@LitUpMagazine1 @thomaschattwill @nhannahjones Well, there is an argument to be made for people coming from different backgrounds and how that affects them. Which also mostly explain the inequality better than race: poor people tend to do worse than middle class/rich people and some ethnicities tends to be poorer than others.",1457055229468348420,2021-11-06T18:40:12.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",43090575,Twitter for Android,[1457055229468348420],"[{'type': 'replied_to', 'id': '1457014467821125639'}]",1457008747583266818,en,1216087959503429633,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN
2,"{'mentions': {}, 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}",There's nothing libs love more than talking about institutional problems. Through parliaments one can set up all sort of investigative committees and set proposals. It's a great way to not actually address the economic inequality faced by the black community.,1457055203388280833,2021-11-06T18:40:05.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 2, 'quote_count': 0}",1406772203203579905,Twitter for Android,[1457055203388280833],"[{'type': 'replied_to', 'id': '1457049749392175105'}]",1457049749392175105,en,1406772203203579905,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN
3,"{'mentions': {}, 'urls': [{'start': 34, 'end': 57, 'url': 'https://t.co/MK6E2AiVzq', 'expanded_url': 'https://twitter.com/ProfTimBale/status/1456649171231035395', 'display_url': 'twitter.com/ProfTimBale/stâ¦'}], 'annotations': {}, 'hashtags': {}, 'cashtags': {}}",Inequality has effects everywhere https://t.co/MK6E2AiVzq,1457055130914861057,2021-11-06T18:39:48.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",728219763227410432,Twitter for Android,[1457055130914861057],"[{'type': 'quoted', 'id': '1456649171231035395'}]",1457055130914861057,en,NaN,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN
4,"{'mentions': {}, 'urls': {}, 'annotations': [{'start': 50, 'end': 64, 'probability': 0.7184, 'type': 'Organization', 'normalized_text': 'Equity Alliance'}, {'start': 69, 'end': 77, 'probability': 0.8453, 'type': 'Place', 'normalized_text': 'Minnesota'}], 'hashtags': {}, 'cashtags': {}}","/6-Now, 'equity'-based education being handled by Equity Alliance of Minnesota. I don't know anything about them. I haven't heard why the change. Anyone?\nMy point: Focus changed from ""

In [4]:
len(merged_df)

448227

In [5]:
df = merged_df.iloc[:5]
df

,entities,text,id,created_at,public_metrics,author_id,source,edit_history_tweet_ids,referenced_tweets,conversation_id,lang,in_reply_to_user_id,possibly_sensitive,geo,attachments,withheld
0,"{'mentions': [{'start': 16, 'end': 32, 'username': 'ChrisJo80574828', 'id': '1130338903133528064'}, {'start': 33, 'end': 46, 'username': 'TheRealEarns', 'id': '374410249'}, {'start': 47, 'end': 60, 'username': 'MarinaMedvin', 'id': '986232040021549056'}], 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}","@peanut_pitbull @ChrisJo80574828 @TheRealEarns @MarinaMedvin The government has no money they spend more than they receive via taxes. So they borrow from the fed which worsens inflation,worsens wealth inequality, and inflates bubbles. What good are roads etc if you are priced out of purchasing goods, unable to pay rent, and no savings",1457055231225778177,2021-11-06T18:40:12.000Z,"{'retweet_count': 0, 'reply_count': 2, 'like_count': 0, 'quote_count': 0}",1230697110321815552,Twitter for Android,[1457055231225778177],"[{'type': 'replied_to', 'id': '1457002564323401737'}]",1456831355556352003,en,796109962292449280,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN
1,"{'mentions': [{'start': 0, 'end': 15, 'username': 'LitUpMagazine1', 'id': '1216087959503429633'}, {'start': 16, 'end': 32, 'username': 'thomaschattwill', 'id': '133033883'}, {'start': 33, 'end': 46, 'username': 'nhannahjones', 'id': '25598396'}], 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}","@LitUpMagazine1 @thomaschattwill @nhannahjones Well, there is an argument to be made for people coming from different backgrounds and how that affects them. Which also mostly explain the inequality better than race: poor people tend to do worse than middle class/rich people and some ethnicities tends to be poorer than others.",1457055229468348420,2021-11-06T18:40:12.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",43090575,Twitter for Android,[1457055229468348420],"[{'type': 'replied_to', 'id': '1457014467821125639'}]",1457008747583266818,en,1216087959503429633,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN
2,"{'mentions': {}, 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}",There's nothing libs love more than talking about institutional problems. Through parliaments one can set up all sort of investigative committees and set proposals. It's a great way to not actually address the economic inequality faced by the black community.,1457055203388280833,2021-11-06T18:40:05.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 2, 'quote_count': 0}",1406772203203579905,Twitter for Android,[1457055203388280833],"[{'type': 'replied_to', 'id': '1457049749392175105'}]",1457049749392175105,en,1406772203203579905,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN
3,"{'mentions': {}, 'urls': [{'start': 34, 'end': 57, 'url': 'https://t.co/MK6E2AiVzq', 'expanded_url': 'https://twitter.com/ProfTimBale/status/1456649171231035395', 'display_url': 'twitter.com/ProfTimBale/stâ¦'}], 'annotations': {}, 'hashtags': {}, 'cashtags': {}}",Inequality has effects everywhere https://t.co/MK6E2AiVzq,1457055130914861057,2021-11-06T18:39:48.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 0, 'quote_count': 0}",728219763227410432,Twitter for Android,[1457055130914861057],"[{'type': 'quoted', 'id': '1456649171231035395'}]",1457055130914861057,en,NaN,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN
4,"{'mentions': {}, 'urls': {}, 'annotations': [{'start': 50, 'end': 64, 'probability': 0.7184, 'type': 'Organization', 'normalized_text': 'Equity Alliance'}, {'start': 69, 'end': 77, 'probability': 0.8453, 'type': 'Place', 'normalized_text': 'Minnesota'}], 'hashtags': {}, 'cashtags': {}}","/6-Now, 'equity'-based education being handled by Equity Alliance of Minnesota. I don't know anything about them. I haven't heard why the change. Anyone?\nMy point: Focus changed from ""

In [6]:
def preprocess_text(text):
    # removing URLs
    url_regex = re.compile(r'http\S+')
    text = url_regex.sub('', text)
    
    # expanding contractions
    text = contractions.fix(text)
    
    # removing special characters and digits
    text = re.sub('[^a-zA-Z]', ' ', text)
    
    # converting to lowercase
    text = text.lower()
    
    # tokenizing text
    tokens = word_tokenize(text)
    
    # removing stopwords
    stop_words = stopwords.words('english')
    tokens = [word for word in tokens if word not in stop_words]
    
    # lemmatizing tokens
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # joining tokens back into a single string
    text = ' '.join(tokens)
    
    return text

In [7]:
# Define a regular expression to match URLs
#url_regex = re.compile(r'http\S+')

# Remove URLs from the 'cleaned_text' column
#df['cleaned_text'] = df['text'].apply(lambda x: url_regex.sub('', x))

# Remove stopwords
#stop_words = stopwords.words('english')
#df['cleaned_text'] = df['cleaned_text'].apply(lambda x: ' '.join([word for word in x.split() if word.lower() not in stop_words]))

# Remove punctuation
#df['cleaned_text'] = df['cleaned_text'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [8]:
# Apply preprocessing function to 'text' column of DataFrame
df['cleaned_text'] = df['text'].apply(preprocess_text)

# Tokenize the text in the 'text' column of your dataframe
texts = [[token.text for token in nlp(text)] for text in df['cleaned_text']]

# Create a dictionary of unique words from the tokenized texts
dictionary = Dictionary(texts)

# Convert the tokenized texts to bag-of-words format using the dictionary
corpus = [dictionary.doc2bow(text) for text in texts]

# Train an LDA model on the corpus
lda_model = LdaModel(corpus=corpus, num_topics=10, id2word=dictionary)

In [9]:
def get_dominant_topic(model, corpus, texts):
    # Get main topic in each document
    topic_keywords = model.show_topics(formatted=False)
    dominant_topic = []
    for i, row in enumerate(model[corpus]):
        row = sorted(row, key=lambda x: x[1], reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # get dominant topic
                topic = topic_keywords[topic_num]
                words = [word for word, prop in topic[1]]
                topic_name = ', '.join(words)
                dominant_topic.append(topic_name)
            else:
                break
    return dominant_topic

In [10]:
df['topic'] = get_dominant_topic(lda_model, corpus, df['cleaned_text'])

In [11]:
df

,entities,text,id,created_at,public_metrics,author_id,source,edit_history_tweet_ids,referenced_tweets,conversation_id,lang,in_reply_to_user_id,possibly_sensitive,geo,attachments,withheld,cleaned_text,topic
0,"{'mentions': [{'start': 16, 'end': 32, 'username': 'ChrisJo80574828', 'id': '1130338903133528064'}, {'start': 33, 'end': 46, 'username': 'TheRealEarns', 'id': '374410249'}, {'start': 47, 'end': 60, 'username': 'MarinaMedvin', 'id': '986232040021549056'}], 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}","@peanut_pitbull @ChrisJo80574828 @TheRealEarns @MarinaMedvin The government has no money they spend more than they receive via taxes. So they borrow from the fed which worsens inflation,worsens wealth inequality, and inflates bubbles. What good are roads etc if you are priced out of purchasing goods, unable to pay rent, and no savings",1457055231225778177,2021-11-06T18:40:12.000Z,"{'retweet_count': 0, 'reply_count': 2, 'like_count': 0, 'quote_count': 0}",1230697110321815552,Twitter for Android,[1457055231225778177],"[{'type': 'replied_to', 'id': '1457002564323401737'}]",1456831355556352003,en,796109962292449280,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN,peanut pitbull chrisjo therealearns marinamedvin government money spend receive via tax borrow fed worsens inflation worsens wealth inequality inflates bubble good road etc priced purchasing good unable pay rent saving,"inequality, good, worsens, set, everywhere, effect, unable, priced, via, bubble"
1,"{'mentions': [{'start': 0, 'end': 15, 'username': 'LitUpMagazine1', 'id': '1216087959503429633'}, {'start': 16, 'end': 32, 'username': 'thomaschattwill', 'id': '133033883'}, {'start': 33, 'end': 46, 'username': 'nhannahjones', 'id': '25598396'}], 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}","@LitUpMagazine1 @thomaschattwill @nhannahjones Well, there is an argument to be made for people coming from different backgrounds and how that affects them. Which also mostly explain the inequality better than race: poor people tend to do worse than middle class/rich people and some ethnicities tends to be poorer than others.",1457055229468348420,2021-11-06T18:40:12.000Z,"{'retweet_count': 0, 'reply_count': 1, 'like_count': 0, 'quote_count': 0}",43090575,Twitter for Android,[1457055229468348420],"[{'type': 'replied_to', 'id': '1457014467821125639'}]",1457008747583266818,en,1216087959503429633,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN,litupmagazine thomaschattwill nhannahjones well argument made people coming different background affect also mostly explain inequality better race poor people tend worse middle class rich people ethnicity tends poorer others,"people, inequality, rich, made, middle, worse, also, well, coming, tend"
2,"{'mentions': {}, 'urls': {}, 'annotations': {}, 'hashtags': {}, 'cashtags': {}}",There's nothing libs love more than talking about institutional problems. Through parliaments one can set up all sort of investigative committees and set proposals. It's a great way to not actually address the economic inequality faced by the black community.,1457055203388280833,2021-11-06T18:40:05.000Z,"{'retweet_count': 0, 'reply_count': 0, 'like_count': 2, 'quote_count': 0}",1406772203203579905,Twitter for Android,[1457055203388280833],"[{'type': 'replied_to', 'id': '1457049749392175105'}]",1457049749392175105,en,1406772203203579905,False,{'coordinates': {'coordinates': {}}},{'media_keys': {}},NaN,nothing libs love talking institutional problem parliament one set sort investigative committee set proposal great way actually address economic inequality faced black community,"inequality, good, worsens, set, everywhere, effect, unable, priced, via, bubble"
3,"{'mentions': {}, 'urls': [{'start': 34, 'end': 57, 'url': 'https://t.co/MK6E2AiVzq', 'expanded_url': 'https://twitter.com/ProfTimBale/status/1456649171231035395', 'display_url': 'twitter.com/ProfTimBale/stâ¦'}], 'annotations': {}, 'hashtags': {}, '

In [12]:
df.to_csv('sample_data.csv', index=False)